### Import Libraries

In [53]:
import os
os.environ['OPENAI_API_KEY'] = ""

In [55]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [63]:
def process_document(path, chunk_size=1000, chunk_overlap=200):
    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    # Create embeddings and vector store
    embeddings  = SentenceTransformerEmbeddings(model_name = "BAAI/bge-small-en-v1.5")
    vector_retriever = FAISS.from_documents(texts, embeddings)
    keyword_retriever = BM25Retriever.from_documents(texts)
    keyword_retriever.k= 3
    return vector_retriever , keyword_retriever

In [64]:
path = "/home/erginous/Desktop/attention.pdf"

In [72]:
vector_retriever , keyword_retriever = process_document(path)
vectorstore_retreiver = vector_retriever.as_retriever(search_kwargs={"k": 3})

In [76]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retreiver,
                                                   keyword_retriever],
                                       weights=[0.3, 0.7])

In [77]:
docs = ensemble_retriever.invoke("what is positional encoding ?")

In [78]:
from pydantic import BaseModel , Field
class scoring(BaseModel):
    score: float = Field(... ,description="The relevance score of a document corresponding to the query")

def rerank(query , docs , top_n=3):
    prompt_template = PromptTemplate(
        input_variables=["query", "doc"],
        template="""On a scale of 1-10, rate the relevance of the following document to the query. Consider the specific context and intent of the query, not just keyword matches.
        Query: {query}
        Document: {doc}
        Relevance Score:"""
    )
    
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)
    llm_chain = prompt_template | llm.with_structured_output(scoring)
    
    scored_docs = []
    for doc in docs:
        input_data = {"query": query, "doc": doc.page_content}
        score = llm_chain.invoke(input_data).score
        try:
            score = float(score)
        except ValueError:
            score = 0  # Default score if parsing fails
        scored_docs.append((doc, score))
    
    reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in reranked_docs[:top_n]]

In [80]:
query = "what is Positional Encoding ?"
reranked_docs = rerank(query, docs)

In [81]:
docs

[Document(metadata={'source': '/home/erginous/Desktop/attention.pdf', 'page': 5}, page_content='tokens in the sequence. To this end, we add "positional encodings" to the input embeddings at the\nbottoms of the encoder and decoder stacks. The positional encodings have the same dimension dmodel\nas the embeddings, so that the two can be summed. There are many choices of positional encodings,\nlearned and fixed [9].\nIn this work, we use sine and cosine functions of different frequencies:\nPE(pos,2i)=sin(pos/100002i/d model)\nPE(pos,2i+1)=cos(pos/100002i/d model)\nwhere posis the position and iis the dimension. That is, each dimension of the positional encoding\ncorresponds to a sinusoid. The wavelengths form a geometric progression from 2πto10000 ·2π. We\nchose this function because we hypothesized it would allow the model to easily learn to attend by\nrelative positions, since for any fixed offset k,PEpos+kcan be represented as a linear function of\nPEpos.\nWe also experimented with usi

In [82]:
reranked_docs

[Document(metadata={'source': '/home/erginous/Desktop/attention.pdf', 'page': 5}, page_content='tokens in the sequence. To this end, we add "positional encodings" to the input embeddings at the\nbottoms of the encoder and decoder stacks. The positional encodings have the same dimension dmodel\nas the embeddings, so that the two can be summed. There are many choices of positional encodings,\nlearned and fixed [9].\nIn this work, we use sine and cosine functions of different frequencies:\nPE(pos,2i)=sin(pos/100002i/d model)\nPE(pos,2i+1)=cos(pos/100002i/d model)\nwhere posis the position and iis the dimension. That is, each dimension of the positional encoding\ncorresponds to a sinusoid. The wavelengths form a geometric progression from 2πto10000 ·2π. We\nchose this function because we hypothesized it would allow the model to easily learn to attend by\nrelative positions, since for any fixed offset k,PEpos+kcan be represented as a linear function of\nPEpos.\nWe also experimented with usi